# 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn.neighbors import BallTree
from haversine import haversine, Unit

# 데이터 불러오기

In [2]:
data_path: str = "~/house/data"
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [3]:
# 금리, 지하철, 학교, 공원 정보 불러오기
interest_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
subway_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))
school_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
park_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))

In [4]:
train_data.head()

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0


In [5]:
test_data.head()

,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age
0,0,84.961,202404,12,1,14,2016,36.965423,127.048779,8
1,1,59.900,202404,13,0,4,1997,36.963105,127.040678,27
2,2,39.270,202404,29,0,5,1990,36.957089,127.047449,34
3,3,39.270,202405,3,0,1,1990,36.957089,127.047449,34
4,4,46.980,202406,2,0,4,1990,36.957089,127.047449,34


In [6]:
interest_data

,year_month,interest_rate
0,202405,3.56
1,202404,3.54
2,202403,3.59
3,202402,3.62
4,202401,3.66
...,...,...
61,201904,1.85
62,201903,1.94
63,201902,1.92
64,201901,1.99


## 데이터 병합하기

### interestRate

In [7]:
merged_train = pd.merge(train_data, interest_data, left_on='contract_year_month', right_on='year_month', how='left')
merged_train = merged_train.drop(columns=['year_month'])
print(merged_train.head())

   index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0  84.9981               201906            25              2      9   
1      1  84.9981               202003            26              2     20   
2      2  84.9981               202003            28              2      8   
3      3  59.3400               201907            15              2      1   
4      4  59.8100               201904            12              2      6   

   built_year   latitude   longitude  age  deposit  interest_rate  
0        2019  37.054314  127.045216    0  17000.0           1.78  
1        2019  37.054314  127.045216    1  23000.0           1.26  
2        2019  37.054314  127.045216    1  23000.0           1.26  
3        1986  36.964647  127.055847   33   5000.0           1.68  
4        1995  36.972390  127.084514   24   1800.0           1.85  


In [8]:
merged_test = pd.merge(test_data, interest_data, left_on='contract_year_month', right_on='year_month', how='left')
merged_test = merged_test.drop(columns=['year_month'])
print(merged_test.head())

   index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0   84.961               202404            12              1     14   
1      1   59.900               202404            13              0      4   
2      2   39.270               202404            29              0      5   
3      3   39.270               202405             3              0      1   
4      4   46.980               202406             2              0      4   

   built_year   latitude   longitude  age  interest_rate  
0        2016  36.965423  127.048779    8           3.54  
1        1997  36.963105  127.040678   27           3.54  
2        1990  36.957089  127.047449   34           3.54  
3        1990  36.957089  127.047449   34           3.56  
4        1990  36.957089  127.047449   34            NaN  


### sklearn의 BallTree를 활용한 haversine 거리 계산 방식

In [9]:
def find_closest_distance_haversine(train_data, loc_df):
    # degree->radian 값으로 변환 for 삼각함수
    train_coords = np.radians(train_data[['latitude', 'longitude']].values)
    loc_coords = np.radians(loc_df[['latitude', 'longitude']].values)

    tree = BallTree(loc_coords, metric='haversine') # Ball Tree 생성 

    distances, indices = tree.query(train_coords, k=1) #가까운 1 지점만
    distances_meter = distances * 6371000 # meter값으로 변환

    closest_coords = loc_df[['latitude', 'longitude']].iloc[indices.flatten()].values # 가까운 지점 좌표

    result_df = pd.DataFrame({
        'index' : train_data.index,
        'closest_distance' : distances_meter.flatten(),
        'closest_latitude' : closest_coords[:, 0],
        'closest_longtitude' : closest_coords[:, 1]
    })

    return result_df


### subway 병합

In [10]:
subway_result = find_closest_distance_haversine(train_data, subway_data)
subway_result.columns = ['index', 'nearest_subway_distance', 'nearest_subway_latitude', 'nearest_subway_longtitude']
train_data = pd.merge(train_data, subway_result, on='index')

In [11]:
subway_result = find_closest_distance_haversine(test_data, subway_data)
subway_result.columns = ['index', 'nearest_subway_distance', 'nearest_subway_latitude', 'nearest_subway_longtitude']
test_data = pd.merge(test_data, subway_result, on='index')

### school 병합

In [12]:
school_result = find_closest_distance_haversine(train_data, school_data)
school_result.columns = ['index', 'nearest_school_distance', 'nearest_school_latitude', 'nearest_school_longtitude']
train_data = pd.merge(train_data, school_result, on='index')

In [13]:
school_result = find_closest_distance_haversine(test_data, school_data)
school_result.columns = ['index', 'nearest_school_distance', 'nearest_school_latitude', 'nearest_school_longtitude']
test_data = pd.merge(test_data, school_result, on='index')

### park 병합

In [14]:
park_result = find_closest_distance_haversine(train_data, park_data)
park_result.columns = ['index', 'nearest_park_distance', 'nearest_park_latitude', 'nearest_park_longtitude']
train_data = pd.merge(train_data, park_result, on='index')

In [15]:
park_result = find_closest_distance_haversine(test_data, park_data)
park_result.columns = ['index', 'nearest_park_distance', 'nearest_park_latitude', 'nearest_park_longtitude']
test_data = pd.merge(test_data, park_result, on='index')

In [16]:
train_data = train_data.drop(columns=['index'])
train_data.head()

,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,nearest_subway_distance,nearest_subway_latitude,nearest_subway_longtitude,nearest_school_distance,nearest_school_latitude,nearest_school_longtitude,nearest_park_distance,nearest_park_latitude,nearest_park_longtitude
0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0,716.952948,37.056496,127.052819,156.120431,37.053232,127.046337,498.618918,37.051333,127.041019
1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0,716.952948,37.056496,127.052819,156.120431,37.053232,127.046337,498.618918,37.051333,127.041019
2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0,716.952948,37.056496,127.052819,156.120431,37.053232,127.046337,498.618918,37.051333,127.041019
3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0,3897.279708,36.990726,127.085159,214.559689,36.962943,127.056980,169.839678,36.963502,127.054582
4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0,2039.685349,36.990726,127.085159,1708.489263,36.987746,127.085154,382.401815,36.971743,127.088742


In [17]:
test_data = test_data.drop(columns=['index'])
test_data.head()

,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,nearest_subway_distance,nearest_subway_latitude,nearest_subway_longtitude,nearest_school_distance,nearest_school_latitude,nearest_school_longtitude,nearest_park_distance,nearest_park_latitude,nearest_park_longtitude
0,84.961,202404,12,1,14,2016,36.965423,127.048779,8,4284.771362,36.990726,127.085159,779.057020,36.962943,127.056980,288.442840,36.964653,127.045679
1,59.900,202404,13,0,4,1997,36.963105,127.040678,27,5004.558790,36.990726,127.085159,1448.494091,36.962943,127.056980,153.733042,36.961730,127.040502
2,39.270,202404,29,0,5,1990,36.957089,127.047449,34,5021.183886,36.990726,127.085159,808.416163,36.950687,127.051762,272.286038,36.959505,127.047945
3,39.270,202405,3,0,1,1990,36.957089,127.047449,34,5021.183886,36.990726,127.085159,808.416163,36.950687,127.051762,272.286038,36.959505,127.047945
4,46.980,202406,2,0,4,1990,36.957089,127.047449,34,5021.183886,36.990726,127.085159,808.416163,36.950687,127.051762,272.286038,36.959505,127.047945
